In [1]:
import time
import datetime
# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

start_time = time.time()
print("  Training epoch took: {:}".format(format_time(time.time() - start_time)))

  Training epoch took: 0:00:00


In [2]:
"""Training IGMC model on the MovieLens dataset."""

import os
import sys
import time
import glob
import random
import argparse
from shutil import copy

import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim

from model import IGMC
from data_rotten import RottenTomato
from dataset_rotten import RottenTomatoDataset, collate_rotten_tomato
from utils import MetricLogger

Using backend: pytorch


In [3]:
def evaluate(model, loader, device):
    # Evaluate RMSE
    model.eval()
    mse = 0.
    for batch in loader:
        with th.no_grad():
            preds = model(batch[0].to(device))
        labels = batch[1].to(device)
        mse += ((preds - labels) ** 2).sum().item()
    mse /= len(loader.dataset)
    return np.sqrt(mse)

def adj_rating_reg(model):
    arr_loss = 0
    for conv in model.convs:
        weight = conv.weight.view(conv.num_bases, conv.in_feat * conv.out_feat)
        weight = th.matmul(conv.w_comp, weight).view(conv.num_rels, conv.in_feat, conv.out_feat)
        arr_loss += th.sum((weight[1:, :, :] - weight[:-1, :, :])**2)
    return arr_loss

# @profile
def train_epoch(model, loss_fn, optimizer, arr_lambda, loader, device, log_interval):
    model.train()

    epoch_loss = 0.
    iter_loss = 0.
    iter_mse = 0.
    iter_cnt = 0
    iter_dur = []

    for iter_idx, batch in enumerate(loader, start=1):
        t_start = time.time()

        inputs = batch[0].to(device)
        labels = batch[1].to(device)
        preds = model(inputs)
        loss = loss_fn(preds, labels).mean() + arr_lambda * adj_rating_reg(model)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * preds.shape[0]
        iter_loss += loss.item() * preds.shape[0]
        iter_mse += ((preds - labels) ** 2).sum().item()
        iter_cnt += preds.shape[0]
        iter_dur.append(time.time() - t_start)

        if iter_idx % log_interval == 0:
            print("Iter={}, loss={:.4f}, mse={:.4f}, time={:.4f}".format(
                iter_idx, iter_loss/iter_cnt, iter_mse/iter_cnt, np.average(iter_dur)))
            iter_loss = 0.
            iter_mse = 0.
            iter_cnt = 0

    return epoch_loss / len(loader.dataset)

def train(args):
    movielens = MovieLens(args.data_name, testing=args.testing,
                            test_ratio=args.data_test_ratio, valid_ratio=args.data_valid_ratio)
    if args.testing:
        test_dataset = RottenTomatoDataset(
            movielens.test_rating_pairs, movielens.test_rating_values, movielens.train_graph, 
            args.hop, args.sample_ratio, args.max_nodes_per_hop) 
    else:
        test_dataset = RottenTomatoDataset(
            movielens.valid_rating_pairs, movielens.valid_rating_values, movielens.train_graph, 
            args.hop, args.sample_ratio, args.max_nodes_per_hop)
    train_dataset = RottenTomatoDataset(
        movielens.train_rating_pairs, movielens.train_rating_values, movielens.train_graph, 
        args.hop, args.sample_ratio, args.max_nodes_per_hop)

    train_loader = th.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, 
                            num_workers=args.num_workers, collate_fn=collate_movielens)
    test_loader = th.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, 
                            num_workers=args.num_workers, collate_fn=collate_movielens)

    in_feats = (args.hop+1)*2 #+ movielens.train_graph.ndata['refex'].shape[1]
    model = IGMC(in_feats=in_feats, 
                 latent_dim=[32, 32, 32, 32],
                 num_relations=5, # movielens.num_rating, 
                 num_bases=4, 
                 regression=True, 
                 edge_dropout=args.edge_dropout,
                #  side_features=args.use_features,
                #  n_side_features=n_features,
                #  multiply_by=args.multiply_by
            ).to(args.device)
    loss_fn = nn.MSELoss().to(args.device)
    optimizer = optim.Adam(model.parameters(), lr=args.train_lr, weight_decay=0)
    print("Loading network finished ...\n")

    ### prepare the logger
    logger = MetricLogger(args.save_dir, args.valid_log_interval)
    
    best_epoch = 0
    best_rmse = np.inf
    ### declare the loss information
    print("Start training ...")
    for epoch_idx in range(1, args.train_epochs+1):
        print ('Epoch', epoch_idx)
    
        train_loss = train_epoch(model, loss_fn, optimizer, args.arr_lambda, 
                                train_loader, args.device, args.train_log_interval)
        test_rmse = evaluate(model, test_loader, args.device)
        eval_info = {
            'epoch': epoch_idx,
            'train_loss': train_loss,
            'test_rmse': test_rmse,
        }
        print('=== Epoch {}, train loss {:.6f}, test rmse {:.6f} ==='.format(*eval_info.values()))

        if epoch_idx % args.train_lr_decay_step == 0:
            for param in optimizer.param_groups:
                param['lr'] = args.train_lr_decay_factor * param['lr']

        logger.log(eval_info, model, optimizer)
        if best_rmse > test_rmse:
            best_rmse = test_rmse
            best_epoch = epoch_idx
    eval_info = "Training ends. The best testing rmse is {:.6f} at epoch {}".format(best_rmse, best_epoch)
    print(eval_info)
    with open(os.path.join(args.save_dir, 'log.txt'), 'a') as f:
        f.write(eval_info)

In [4]:
def config():
    parser = argparse.ArgumentParser(description='IGMC')
    # general settings
    parser.add_argument('--testing', action='store_true', default=False,
                        help='if set, use testing mode which splits all ratings into train/test;\
                        otherwise, use validation model which splits all ratings into \
                        train/val/test and evaluate on val only')
    parser.add_argument('--device', default='0', type=int,
                        help='Running device. E.g `--device 0`, if using cpu, set `--device -1`')
    parser.add_argument('--seed', type=int, default=1234, metavar='S',
                        help='random seed (default: 1234)')
    parser.add_argument('--data_name', default='ml-100k', type=str,
                        help='The dataset name: ml-100k, ml-1m')
    parser.add_argument('--data_test_ratio', type=float, default=0.1) # for ml-100k the test ration is 0.2
    parser.add_argument('--num_workers', type=int, default=8)
    parser.add_argument('--data_valid_ratio', type=float, default=0.2)
    # parser.add_argument('--ensemble', action='store_true', default=False,
    #                     help='if True, load a series of model checkpoints and ensemble the results')               
    parser.add_argument('--train_log_interval', type=int, default=100)
    parser.add_argument('--valid_log_interval', type=int, default=10)
    parser.add_argument('--save_appendix', type=str, default='debug', 
                        help='what to append to save-names when saving results')
    # subgraph extraction settings
    parser.add_argument('--hop', default=1, metavar='S', 
                        help='enclosing subgraph hop number')
    parser.add_argument('--sample_ratio', type=float, default=1.0, 
                        help='if < 1, subsample nodes per hop according to the ratio')
    parser.add_argument('--max_nodes_per_hop', type=int, default=200, 
                        help='if > 0, upper bound the # nodes per hop by another subsampling')
    # parser.add_argument('--use_features', action='store_true', default=False,
    #                     help='whether to use node features (side information)')
    # edge dropout settings
    parser.add_argument('--edge_dropout', type=float, default=0.2, 
                        help='if not 0, random drops edges from adjacency matrix with this prob')
    parser.add_argument('--force_undirected', action='store_true', default=False, 
                        help='in edge dropout, force (x, y) and (y, x) to be dropped together')
    # optimization settings
    parser.add_argument('--train_lr', type=float, default=1e-3)
    parser.add_argument('--train_min_lr', type=float, default=1e-6)
    parser.add_argument('--train_lr_decay_factor', type=float, default=0.1)
    parser.add_argument('--train_lr_decay_step', type=int, default=50)
    parser.add_argument('--train_epochs', type=int, default=80)
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--arr_lambda', type=float, default=0.001)
    parser.add_argument('--num_rgcn_bases', type=int, default=4)
                
    args = parser.parse_args()
    args.device = th.device(args.device) if args.device >= 0 and th.cuda.is_available() else th.device('cpu')
    
    ### set save_dir according to localtime and test mode
    file_dir = os.path.dirname(os.path.realpath('__file__'))
    val_test_appendix = 'testmode' if args.testing else 'valmode'
    local_time = time.strftime('%y%m%d%H%M', time.localtime())
    args.save_dir = os.path.join(
        file_dir, 'log/{}_{}_{}_{}'.format(
            args.data_name, args.save_appendix, val_test_appendix, local_time
        )
    )
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir) 
    print(args)

    # backup current .py files
    for f in glob.glob(r"*.py"):
        copy(f, args.save_dir)

    # save command line input
    cmd_input = 'python3 ' + ' '.join(sys.argv)
    with open(os.path.join(args.save_dir, 'cmd_input.txt'), 'a') as f:
        f.write(cmd_input)
        f.write("\n")
    print('Command line input: ' + cmd_input + ' is saved.')
    
    return args

In [5]:
# if __name__ == '__main__':
#     args = config()
#     random.seed(args.seed)
#     np.random.seed(args.seed)
#     th.manual_seed(args.seed)
#     if th.cuda.is_available():
#         th.cuda.manual_seed_all(args.seed)
#     train(args)

## 1. Config

In [6]:
import easydict

args = easydict.EasyDict({ 
    'data_name':            'rotten',
    'testing':     	        True,
    'device':      	        0,
    'seed':        	        1234,
    'data_test_ratio':      0.1,
    'num_workers':   	    8,
    'data_valid_ratio':     0.2,
    'train_log_interval':   200,
    'valid_log_interval':   10,
    'save_appendix':   	    'debug',
    'hop':   	            1,
    'sample_ratio':    	    1.0,
    'max_nodes_per_hop':    100,
    'edge_dropout':   	    0.2,
    'force_undirected':     False,
    'train_lr':   	        1e-3,
    'train_min_lr':   	    1e-6,
    'train_lr_decay_factor':0.1,
    'train_lr_decay_step':  50,
    'train_epochs':   	    1,
    'batch_size':   	    32,
    'arr_lambda':   	    0.001,
    'num_rgcn_bases':   	4,
    'train_epochs':   	    1
})

In [7]:
### set save_dir according to localtime and test mode
file_dir = os.path.dirname(os.path.realpath('__file__'))
val_test_appendix = 'testmode' if args.testing else 'valmode'
local_time = time.strftime('%y%m%d%H%M', time.localtime())
args.save_dir = os.path.join(
    file_dir, 'log/{}_{}_{}_{}'.format(
        args.data_name, args.save_appendix, val_test_appendix, local_time
    )
)
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir) 
print(args)

# backup current .py files
for f in glob.glob(r"*.py"):
    copy(f, args.save_dir)

# save command line input
cmd_input = 'python3 ' + ' '.join(sys.argv)
with open(os.path.join(args.save_dir, 'cmd_input.txt'), 'a') as f:
    f.write(cmd_input)
    f.write("\n")
print('Command line input: ' + cmd_input + ' is saved.')

{'data_name': 'rotten', 'testing': True, 'device': 0, 'seed': 1234, 'data_test_ratio': 0.1, 'num_workers': 8, 'data_valid_ratio': 0.2, 'train_log_interval': 200, 'valid_log_interval': 10, 'save_appendix': 'debug', 'hop': 1, 'sample_ratio': 1.0, 'max_nodes_per_hop': 100, 'edge_dropout': 0.2, 'force_undirected': False, 'train_lr': 0.001, 'train_min_lr': 1e-06, 'train_lr_decay_factor': 0.1, 'train_lr_decay_step': 50, 'train_epochs': 1, 'batch_size': 32, 'arr_lambda': 0.001, 'num_rgcn_bases': 4, 'save_dir': 'C:\\Users\\user\\Jupyter_project\\keejun\\graph\\IGMC_kirc\\log/rotten_debug_testmode_2111112152'}
Command line input: python3 C:\Users\user\anaconda3\envs\graph\lib\site-packages\ipykernel_launcher.py -f C:\Users\user\AppData\Roaming\jupyter\runtime\kernel-06857397-42d0-4553-9b07-b1e2f1697dfd.json is saved.


In [8]:
args

{'data_name': 'rotten',
 'testing': True,
 'device': 0,
 'seed': 1234,
 'data_test_ratio': 0.1,
 'num_workers': 8,
 'data_valid_ratio': 0.2,
 'train_log_interval': 200,
 'valid_log_interval': 10,
 'save_appendix': 'debug',
 'hop': 1,
 'sample_ratio': 1.0,
 'max_nodes_per_hop': 100,
 'edge_dropout': 0.2,
 'force_undirected': False,
 'train_lr': 0.001,
 'train_min_lr': 1e-06,
 'train_lr_decay_factor': 0.1,
 'train_lr_decay_step': 50,
 'train_epochs': 1,
 'batch_size': 32,
 'arr_lambda': 0.001,
 'num_rgcn_bases': 4,
 'save_dir': 'C:\\Users\\user\\Jupyter_project\\keejun\\graph\\IGMC_kirc\\log/rotten_debug_testmode_2111112152'}

In [9]:
random.seed(args.seed)
np.random.seed(args.seed)
th.manual_seed(args.seed)
if th.cuda.is_available():
    th.cuda.manual_seed_all(args.seed)    

In [10]:
# start_time = time.time()

# train(args)

# print("  Training epoch took: {:}".format(format_time(time.time() - start_time)))

## 2. Train

In [11]:
### prepare data and set model
path = './raw_data/rotten_tomato/'
rotten_tomato = RottenTomato(path, testing=args.testing,test_ratio=args.data_test_ratio, valid_ratio=args.data_valid_ratio)

Create RottenTomato Class...
	Train rating pairs : 178320
	Valid rating pairs : 35664
	Test rating pairs  : 26851


In [12]:
rotten_tomato

In [13]:
args.testing

True

In [14]:
if args.testing:
    test_dataset = RottenTomatoDataset(
        rotten_tomato.test_rating_pairs, rotten_tomato.test_rating_values, rotten_tomato.train_graph, 
        args.hop, args.sample_ratio, args.max_nodes_per_hop) 
else:
    test_dataset = RottenTomatoDataset(
        rotten_tomato.valid_rating_pairs, rotten_tomato.valid_rating_values, rotten_tomato.train_graph, 
        args.hop, args.sample_ratio, args.max_nodes_per_hop)

In [15]:
train_dataset = RottenTomatoDataset(
    rotten_tomato.train_rating_pairs, rotten_tomato.train_rating_values, rotten_tomato.train_graph, 
    args.hop, args.sample_ratio, args.max_nodes_per_hop)

In [16]:
train_loader = th.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, 
                        num_workers=args.num_workers, collate_fn=collate_rotten_tomato)
test_loader = th.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, 
                        num_workers=args.num_workers, collate_fn=collate_rotten_tomato)

In [17]:
in_feats = (args.hop+1)*2 #+ rotten_tomato.train_graph.ndata['refex'].shape[1]
model = IGMC(in_feats=in_feats, 
             latent_dim=[32, 32, 32, 32],
             num_relations=5, # rotten_tomato.num_rating, 
             num_bases=4, 
             regression=True, 
             edge_dropout=args.edge_dropout,
            #  side_features=args.use_features,
            #  n_side_features=n_features,
            #  multiply_by=args.multiply_by
        ).to(args.device)
loss_fn = nn.MSELoss().to(args.device)
optimizer = optim.Adam(model.parameters(), lr=args.train_lr, weight_decay=0)
print("Loading network finished ...\n")

Loading network finished ...



In [18]:
args.hop

1

In [19]:
in_feats

4

In [20]:
### prepare the logger
logger = MetricLogger(args.save_dir, args.valid_log_interval)

best_epoch = 0
best_rmse = np.inf
### declare the loss information
print("Start training ...")

Start training ...


In [21]:
start_time = time.time()

for epoch_idx in range(1, args.train_epochs+1):
    print ('Epoch', epoch_idx)

    train_loss = train_epoch(model, loss_fn, optimizer, args.arr_lambda, 
                            train_loader, args.device, args.train_log_interval)
    test_rmse = evaluate(model, test_loader, args.device)
    eval_info = {
        'epoch': epoch_idx,
        'train_loss': train_loss,
        'test_rmse': test_rmse,
    }
    print('=== Epoch {}, train loss {:.6f}, test rmse {:.6f} ==='.format(*eval_info.values()))

    if epoch_idx % args.train_lr_decay_step == 0:
        for param in optimizer.param_groups:
            param['lr'] = args.train_lr_decay_factor * param['lr']

    logger.log(eval_info, model, optimizer)
    if best_rmse > test_rmse:
        best_rmse = test_rmse
        best_epoch = epoch_idx

print("  Training epoch took: {:}".format(format_time(time.time() - start_time)))

Epoch 1
Iter=200, loss=4.5522, mse=4.5504, time=0.0478
Iter=400, loss=3.5049, mse=3.5029, time=0.0431
Iter=600, loss=3.5270, mse=3.5249, time=0.0417
Iter=800, loss=3.3964, mse=3.3941, time=0.0408
Iter=1000, loss=3.4752, mse=3.4728, time=0.0403
Iter=1200, loss=3.4616, mse=3.4590, time=0.0399
Iter=1400, loss=3.2917, mse=3.2891, time=0.0397
Iter=1600, loss=3.3451, mse=3.3423, time=0.0395
Iter=1800, loss=3.3051, mse=3.3022, time=0.0394
Iter=2000, loss=3.4012, mse=3.3982, time=0.0393
Iter=2200, loss=3.3259, mse=3.3228, time=0.0393
Iter=2400, loss=3.1612, mse=3.1580, time=0.0392
Iter=2600, loss=3.3353, mse=3.3320, time=0.0391
Iter=2800, loss=3.3132, mse=3.3098, time=0.0391
Iter=3000, loss=3.2377, mse=3.2342, time=0.0390
Iter=3200, loss=3.2108, mse=3.2072, time=0.0390
Iter=3400, loss=3.2462, mse=3.2426, time=0.0391
Iter=3600, loss=3.1450, mse=3.1413, time=0.0390
Iter=3800, loss=3.2914, mse=3.2875, time=0.0390
Iter=4000, loss=3.2469, mse=3.2431, time=0.0390
Iter=4200, loss=3.2484, mse=3.2444, 

In [22]:
eval_info = "Training ends. The best testing rmse is {:.6f} at epoch {}".format(best_rmse, best_epoch)
print(eval_info)
with open(os.path.join(args.save_dir, 'log.txt'), 'a') as f:
    f.write(eval_info)

Training ends. The best testing rmse is 1.898806 at epoch 1


- IGMC의 users, items 확인하기

In [23]:
model.block

Graph(num_nodes=242, num_edges=1298,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'nlabel': Scheme(shape=(4,), dtype=torch.float32), 'x': Scheme(shape=(4,), dtype=torch.float32)}
      edata_schemes={'etype': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64), 'edge_mask': Scheme(shape=(), dtype=torch.float32)})

In [24]:
model.block.ndata

{'_ID': tensor([ 711,  517,  278,  493,  548,  418,  444,   96,  249,  698,  154,  425,
         323,  675,  822,  296,  210,  522,  401,  186,  416,  466,  385,  261,
         687,  433,  701,  304,  700,  671,  341,  362,   60,  610,  136,  247,
         108,  311,  779,  733,  562,  223,  107,  714,   18,  841,  688,  215,
         497,  685,  455,  553,  690,  620,  326,  431,  113,  629,  736,  706,
         827,  360,  140,  782,  189,  719,  366,  348,  267,   59,  482,  386,
         657,   97,  231,  409,  550,   98,  235,  308,  321,  626,   46,  697,
         391,  317,  850,  410,  823,  798,  130,  786,  300,  393,  659,  357,
         518,  141,  774,  381,  543, 5527, 5582,  181,  415,  455,  575,  594,
         686,  700,  729,  737, 1626, 5279, 7234, 4896, 1825, 6842, 2769, 8564,
        2349, 2518, 4837, 7503, 2205, 3452, 2729, 5383, 3992, 4854, 4818, 6152,
        2886, 2202, 5941, 8462, 5708, 7862, 1785, 4635, 4060, 1992, 8640, 7104,
        5113, 8361, 7634, 2792, 

In [25]:
model.block.ndata['nlabel']

tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0.,

In [26]:
model.block.ndata['nlabel'].shape

torch.Size([242, 4])

In [27]:
model.block.ndata['nlabel'][:, 0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [28]:
model.block_x

tensor([[-0.2057, -0.9095,  0.8907,  ..., -0.9173,  0.7441,  0.9805],
        [-0.1511, -0.8892,  0.8495,  ..., -0.8813,  0.7189,  0.9660],
        [-0.7094, -0.8141,  0.7197,  ..., -0.5087,  0.2565,  0.9730],
        ...,
        [-0.9994, -0.3894, -0.9118,  ...,  0.9001, -0.8125,  0.6751],
        [ 0.3903, -0.7851,  0.4084,  ..., -0.4084,  0.6968,  0.4396],
        [ 0.3903, -0.7851,  0.4084,  ..., -0.4084,  0.6968,  0.4396]],
       device='cuda:0')

In [29]:
model.users.shape

torch.Size([242])

In [30]:
model.items.shape

torch.Size([242])

In [31]:
model.concat_states.shape

torch.Size([242, 128])

In [32]:
model.concat_states

tensor([[-0.6358, -0.2797, -0.6337,  ..., -0.9173,  0.7441,  0.9805],
        [-0.5393, -0.4982, -0.4925,  ..., -0.8813,  0.7189,  0.9660],
        [-0.5468, -0.5067, -0.4928,  ..., -0.5087,  0.2565,  0.9730],
        ...,
        [ 0.0579, -0.1061, -0.1119,  ...,  0.9001, -0.8125,  0.6751],
        [-0.0142, -0.0400, -0.1710,  ..., -0.4084,  0.6968,  0.4396],
        [-0.0142, -0.0400, -0.1710,  ..., -0.4084,  0.6968,  0.4396]],
       device='cuda:0')

In [33]:
model.concat_states[model.users]

tensor([[-0.6358, -0.2797, -0.6337, -0.5025,  0.4910, -0.2071, -0.7283,  0.0339,
          0.6825,  0.4217, -0.1271,  0.6621, -0.4415, -0.2291, -0.3042,  0.4808,
         -0.2599, -0.5215,  0.2434,  0.5987,  0.0722, -0.6885,  0.5980,  0.4211,
         -0.5975, -0.0746, -0.0844,  0.2512,  0.3852,  0.1114,  0.1747,  0.7297,
         -0.6110, -0.4318,  0.4377, -0.4292, -0.7702,  0.8885, -0.8244,  0.6679,
         -0.4753,  0.7979, -0.7246,  0.3284, -0.3571, -0.7500,  0.1002,  0.6705,
          0.5953,  0.8478,  0.7451,  0.8772,  0.4188, -0.3666, -0.8656, -0.1272,
          0.6151, -0.8673,  0.8561,  0.8328,  0.7235,  0.6067,  0.8411,  0.5652,
          0.0814, -0.7639,  0.7591,  0.7039,  0.5658,  0.8375,  0.9523, -0.9454,
          0.9709,  0.9799,  0.8497,  0.6416, -0.8626, -0.9059,  0.7033,  0.7907,
         -0.9101, -0.5427, -0.9290,  0.9404, -0.4945,  0.5848, -0.8710,  0.4651,
         -0.9262, -0.8005,  0.8705,  0.9492, -0.8259, -0.9300,  0.9682,  0.9265,
         -0.2057, -0.9095,  

In [34]:
model.concat_states[model.users].shape

torch.Size([3, 128])

In [35]:
model.concat_states[model.items].shape

torch.Size([3, 128])

In [36]:
concat = th.cat([model.concat_states[model.users], model.concat_states[model.items]], 1)

In [37]:
# user, item vector들을 합침
concat.shape

torch.Size([3, 256])

### Train_epoch 함수 테스트

In [38]:
model = model
loss_fn = loss_fn
optimizer = optimizer
arr_lambda = args.arr_lambda
loader = train_loader
device = args.device
log_interval = args.train_log_interval

In [39]:
log_interval

200

In [40]:
start_time = time.time()

model.train()

epoch_loss = 0.
iter_loss = 0.
iter_mse = 0.
iter_cnt = 0
iter_dur = []

# 서브그래프 단위로 학습
for iter_idx, batch in enumerate(loader, start=1):
    t_start = time.time()

    inputs = batch[0].to(device)
    labels = batch[1].to(device)
    preds = model(inputs)
    loss = loss_fn(preds, labels).mean() + arr_lambda * adj_rating_reg(model)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item() * preds.shape[0]
    iter_loss += loss.item() * preds.shape[0]
    iter_mse += ((preds - labels) ** 2).sum().item()
    iter_cnt += preds.shape[0]
    iter_dur.append(time.time() - t_start)

    if iter_idx % log_interval == 0:
        print("Iter={}, loss={:.4f}, mse={:.4f}, time={:.4f}".format(
            iter_idx, iter_loss/iter_cnt, iter_mse/iter_cnt, np.average(iter_dur)))
        iter_loss = 0.
        iter_mse = 0.
        iter_cnt = 0

train_epoch_loss = epoch_loss / len(loader.dataset)

print("  Time took: {:}".format(format_time(time.time() - start_time)))

Iter=200, loss=3.0832, mse=3.0790, time=0.0394
Iter=400, loss=3.1369, mse=3.1323, time=0.0390
Iter=600, loss=3.2943, mse=3.2898, time=0.0387
Iter=800, loss=3.1302, mse=3.1256, time=0.0388
Iter=1000, loss=3.1018, mse=3.0971, time=0.0387
Iter=1200, loss=3.2010, mse=3.1962, time=0.0386
Iter=1400, loss=3.0876, mse=3.0827, time=0.0385
Iter=1600, loss=3.1327, mse=3.1278, time=0.0385
Iter=1800, loss=3.2319, mse=3.2268, time=0.0385
Iter=2000, loss=3.0849, mse=3.0798, time=0.0386
Iter=2200, loss=3.1549, mse=3.1498, time=0.0385
Iter=2400, loss=3.1133, mse=3.1082, time=0.0385
Iter=2600, loss=3.0773, mse=3.0721, time=0.0385
Iter=2800, loss=3.0440, mse=3.0387, time=0.0385
Iter=3000, loss=3.1351, mse=3.1297, time=0.0385
Iter=3200, loss=3.1349, mse=3.1295, time=0.0386
Iter=3400, loss=2.9213, mse=2.9158, time=0.0388
Iter=3600, loss=3.0939, mse=3.0884, time=0.0388
Iter=3800, loss=3.2014, mse=3.1956, time=0.0388
Iter=4000, loss=3.1302, mse=3.1243, time=0.0388
Iter=4200, loss=2.9488, mse=2.9426, time=0.0

In [41]:
inputs

Graph(num_nodes=2171, num_edges=61386,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'nlabel': Scheme(shape=(4,), dtype=torch.float32), 'x': Scheme(shape=(4,), dtype=torch.float32)}
      edata_schemes={'etype': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64), 'edge_mask': Scheme(shape=(), dtype=torch.float32)})

In [42]:
labels

tensor([7., 9., 7., 6., 7., 3., 6., 3., 5., 4., 9., 5., 0., 3., 3., 6.],
       device='cuda:0')

In [43]:
loss

tensor(3.8209, device='cuda:0', grad_fn=<AddBackward0>)

In [44]:
preds.shape[0]

16

In [45]:
train_epoch_loss

3.0991868031297685

In [46]:
preds

tensor([6.1299, 4.7555, 4.8841, 7.2973, 5.3850, 4.8987, 5.6385, 3.1361, 6.1342,
        3.5716, 6.9831, 4.8422, 4.7172, 2.8222, 4.1561, 6.7424],
       device='cuda:0', grad_fn=<MulBackward0>)

### Evaluate 함수 테스트

In [47]:
model = model
loader = test_loader
device = args.device

In [48]:
start_time = time.time()
predict_ratings = list()
real_ratings = list()

# Evaluate RMSE
model.eval()
mse = 0.
for batch in loader:
    with th.no_grad():
        preds = model(batch[0].to(device))
    labels = batch[1].to(device)
    mse += ((preds - labels) ** 2).sum().item()
    
    real_ratings.append(labels)
    predict_ratings.append(preds)
    
mse /= len(loader.dataset)
rmse = np.sqrt(mse)

print("  Time took: {:}".format(format_time(time.time() - start_time)))

  Time took: 0:00:29


In [49]:
preds

tensor([5.8086, 6.6340, 3.9050], device='cuda:0')

In [50]:
labels

tensor([6., 6., 3.], device='cuda:0')

In [51]:
rmse

1.7701252440065876

In [52]:
test_dataset

In [53]:
predict_ratings[0]

tensor([6.0884, 7.3267, 6.4094, 7.2519, 7.1850, 5.3218, 4.9718, 4.8634, 5.7870,
        7.2103, 4.7904, 6.2181, 7.4301, 4.4984, 5.3829, 4.2108, 4.3310, 6.7832,
        6.8143, 5.5623, 6.2674, 6.1137, 6.4566, 5.4044, 4.6298, 6.1769, 7.2690,
        6.2197, 4.9070, 5.4487, 2.7218, 3.0106], device='cuda:0')

In [54]:
real_ratings[0]

tensor([7., 6., 6., 6., 7., 8., 4., 3., 6., 7., 6., 5., 6., 4., 4., 5., 3., 6.,
        9., 4., 5., 7., 6., 6., 6., 6., 6., 6., 7., 6., 6., 4.],
       device='cuda:0')